# Consulta Alvo da Investigação 

In [42]:
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType
from graphframes import *

In [43]:
# Nomes de aquivos e pastas
checkpoint_dir = 'graphframes_cps_30m'
parquet_dir_name = 'output/embeddings_nome_30m'
parquet_nodes  = 'dataset/grapho/nodes_30m'
parquet_edges = 'dataset/grapho/edges_30m'
# cnpj_alvo='27364903' # Eco em pets
cnpj_alvo='20848420' # Saritur # https://shorturl.at/tEMO7
# cnpj_alvo='15464658' # Sta Terezinha Atacadista # https://shorturl.at/koqz4

## Incicia Secção do Spark

In [44]:
spark = SparkSession.builder \
            .appName("consulta_arvore") \
            .getOrCreate()

sc = spark.sparkContext
sc.setCheckpointDir(checkpoint_dir)

### Realizar a leitura Nodes (Pjs e Pfs) e Edges (relacionamentos) e monta grafo

In [45]:
# Carrega nodes
nodes = spark.read.parquet(parquet_nodes)

In [46]:
# Carrega edges
edges = spark.read.parquet(parquet_edges)

In [47]:
# Carrega embeddings
embeddings = spark.read.parquet(parquet_dir_name)

In [48]:
# Criando o grafo de relacionamento empresas->socios
g = GraphFrame(nodes, edges)

## Começa investigação do CNPJ alvo
O fluxosegue nessa ordem:
1. Consula dos CEP do CNPJ alvo
2. De todas as das empresas no CEP do CNPJ Alvo
3. __Montas-se o Cluster de empresas pelo CEP__ - Leitura dos vetores do embedding dos nomes fantasias do universo de empresas a serem pesquisadas.

In [49]:
# Localiza o CEP do target
cep_do_alvo = g.vertices.filter(f"id = '{cnpj_alvo}'").select("te_dados_es.cep").first()['cep']

In [50]:
print(cep_do_alvo)

30750002


In [51]:
# Localiza todos os CNJPs do cep do alvo
cnpjs_universo = g.vertices.filter(f"te_dados_es.cep = '{cep_do_alvo}'").select("id")

In [52]:
cnpjs_universo.show()

+--------+
|      id|
+--------+
|20848420|
|12640817|
+--------+



In [33]:
# Join com os embeddings para recuperar os vetores
embeddings_universo = cnpjs_universo.join(embeddings, embeddings["id"] == cnpjs_universo["id"], "inner") \
                                    .select(embeddings["id"], embeddings["nomeFantasia"],  embeddings["embeddings"])

### Consulta todos os CNPJ Sócios PJ do CNPJ alvos
Todos os sócios do CNPJ alvo formarão a lista de sociosalvo

In [34]:
# Consulta todos sócios do alvo
id = cnpj_alvo

pFind  = "(a)-[e]->(b)"
socios_target = g.find(pFind) \
          .filter(f"(a.id = '{id}') OR (b.id = '{id}')") \
          .select(col("a.id").alias("src"), col("a.te_dados_es.nomeFantasia").alias("src_nome"),
                  col("b.id").alias("dst"), col("b.te_dados_es.nomeFantasia").alias("dst_nome"),
                  col("b.id_tipoPessoa").alias("id_tipoPessoa")) \
          .distinct()

In [35]:
# Realizar tratamento na pesquisa
condicao_1 = socios_target.select(col('src').alias('id'), col('src_nome').alias('nomeFantasia'))

# Selecionar as linhas onde id_tipoPessoa é diferente de 1
condicao_2 = socios_target.filter(col('id_tipoPessoa') == 1).select(col('dst').alias('id'), col('dst_nome').alias('nomeFantasia'))

# Juntar as duas condições e remover linhas duplicadas
socios_final = condicao_1.union(condicao_2).distinct()

### Procedimenots para montagem da __Matriz de Similariedade__
A Matriz de Similaridade contém pontuações de similaridade entre os nomes nos Sócios Alvos. É usada  similaridade de cosseno que variam de -1 a 1. O valor 1 indica similaridade total e 0 indica nenhuma similaridade. Os passos para sua geração são os segintes:
1. Monta vertor com todos os Nomes dos Socios Alvo
2. Gera os embeddings dos nomes fantasia dos Socios Alvo
3. Monta __Matriz de Similariedade__

In [36]:
# Filtra e seleciona os nomes distintos para cruzamento
nomes_final = socios_final.select("nomeFantasia").distinct().rdd.flatMap(lambda x: x).collect()

In [37]:
# Recuperando embeddings para nomes_combinados
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings_target = model.encode(nomes_final)

In [38]:
# Gera matriz de similiariedade com nomes localizados 
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Converter os embeddings para um formato compatível com o cosine_similarity
embeddings = np.array(embeddings_universo.select("embeddings").collect())
embeddings =  np.vstack(embeddings)

# Calcular a matriz de similaridade usando cosine_similarity
matriz_similaridade = cosine_similarity(embeddings_target, embeddings)

# Imprimir a matriz de similaridade
print(matriz_similaridade)

[[ 0.48437993  0.06112695  0.3877481   0.33208819  0.46445732  0.26643374
   0.45120085  0.49946784  0.53908645  0.37173367  0.23803118  0.3787114
   0.5307848   0.31074607  0.44429191  0.56241698  0.48729338  0.32599433
   0.38956706  0.32954075  0.28852334  0.30830986  0.48192012  0.54464585
   0.44106542  0.24160931  0.41254924  0.32643384  0.33573133  0.49034419
   0.42017717  0.38367666  0.56619518  0.23465743  0.65161641  0.37541271
   0.32955343  0.43044991  0.35533205  0.27959612  0.15069494  0.49034419
   0.34120892  0.37189671  0.28412188  0.31360075  0.31855581  0.3549544
   0.46714585  0.23255178  0.37470291  0.49157648  0.14841119  0.51463363
   0.23348653  0.37582367  0.47618613  0.2532823   0.33836645  0.23673866
   0.43353721  0.4431544   0.29829028  0.22727824  0.43385045  0.34794871
   0.13037715  0.44668107  0.50202164  0.28809648  0.33057877  0.53139432
  -0.02971868  0.40251583  0.39811811  0.4621716   0.38539612  0.40219437
   0.31442478  0.33401158  0.29324184  0

In [39]:
# Localiza coordenadas
ordenadas, abscissas = np.where(matriz_similaridade > 0.9)

In [40]:
# Função para obter os valores de id e nomeFantasia
def obter_valores(dataframe, indices):
    return dataframe.select("id", "nomeFantasia").collect()[indices]
# Imprimir valores encontrados
lista_triplas = []
for i in range(len(ordenadas)):
    # Valor 1: Matriz de similaridade
    valor_1 = matriz_similaridade[ordenadas[i], abscissas[i]]

    # Valor 2: Campos 'id' e 'nomeFantasia' de embeddings_universo usando n-ésimo índice das ordenadas
    valor_2_embeddings = obter_valores(embeddings_universo, abscissas[i])

    # Valor 3: Campos 'id' e 'nomeFantasia' de socios_final usando n-ésimo índice das ordenadas
    valor_3_socios = obter_valores(socios_final, ordenadas[i])

    # Adicionar à lista de triplas
    lista_triplas.append((valor_1, valor_2_embeddings, valor_3_socios))

# Imprimir a lista de triplas
for tripla in lista_triplas:
    print(tripla)

(1.0000000000000004, Row(id='15464658', nomeFantasia='SANTA THEREZINHA ATACADISTA'), Row(id='15464658', nomeFantasia='SANTA THEREZINHA ATACADISTA'))


In [53]:
spark.stop()